In [2]:
%load_ext autoreload
%autoreload 2
# %flow mode reactive

from importlib import reload
from pathlib import Path
import warnings

from dotmap import DotMap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import seaborn as sns
from tqdm import tqdm

import aeon
from aeon.analysis.utils import visits, distancetravelled
from aeon.io import reader
from aeon.io.device import Device, register
from aeon.schema import core, foraging, social
from aeon.schema.schemas import exp02, social01, social02
import datajoint as dj

In [2]:
# Prettify pandas output display.

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_rows", 50)

## Corral some data

In [3]:
"""Create a blocks df."""

exp_start = pd.Timestamp("2024-01-31 00:00:00")
exp_end = pd.Timestamp("2024-02-04 00:00:00")
# roots = [
#     Path("/ceph/aeon/aeon/data/raw/AEON3/social0.2"),
#     Path("/ceph/aeon/aeon/data/raw/AEON4/social0.2"),
# ]
roots = [
    Path(r"Z:\aeon\data\raw\AEON3\social0.2"),
    Path(r"Z:\aeon\data\raw\AEON4\social0.2"),
]
social02.CameraTop.Pose._model_root = Path(r"Z:\aeon\data\processed")
arenas = ["AEON3", "AEON4"]
patches = ["Patch1", "Patch2", "Patch3"]
patch_locs = pd.DataFrame(index=arenas, columns=patches)
blocks_df = pd.DataFrame()
block_ts_tol = pd.Timedelta("2s")  # Tolerance for block start and end times
good_block_pel_ct = 4  # Min pellets for good block


for root in roots:
    # Pull out info from metadata
    for arena in arenas:
        if arena in str(root):
            break
    metadata = (
        aeon.load(root, social02.Metadata, pd.Timestamp("2024-01-31 00:00:00"), exp_end).iloc[0].metadata
    )
    patch_locs.loc[arena, patches] = (
        (metadata.Devices.Patch1Rfid.Location.X, metadata.Devices.Patch1Rfid.Location.Y),
        (metadata.Devices.Patch2Rfid.Location.X, metadata.Devices.Patch2Rfid.Location.Y),
        (metadata.Devices.Patch3Rfid.Location.X, metadata.Devices.Patch3Rfid.Location.Y)
    )
    block_info = aeon.load(root, social02.Environment.BlockState, exp_start, exp_end)

    # Block end if pellet_ct == 0 and preceding pellet_ct > 0
    # OR pellet_ct == 0, preceding pellet_ct == 0, and preceding pellet_ct time diff > 1s
    possible_block_end_indxs = np.where(block_info.pellet_ct == 0)[0]
    drop_indxs = []

    for i in possible_block_end_indxs:
        if block_info.pellet_ct[i - 1] > 0:
            continue
        elif block_info.pellet_ct[i - 1] == 0 and block_info.index[i] - block_info.index[i - 1] > pd.Timedelta("1s"):
            continue
        else:  # drop i from `possible_block_ends`
            drop_indxs.append(i)

    block_end_indxs = np.setdiff1d(possible_block_end_indxs, drop_indxs)
    # Start from first complete block to last complete block
    block_start_indxs = block_end_indxs[0:-1] 
    block_end_indxs = block_end_indxs[1:]
    block_start_times = block_info.index[block_start_indxs] - block_ts_tol
    block_end_times = block_info.index[block_end_indxs] + block_ts_tol
    sleap_model_dir = (
        Path(r"Z:/aeon/data/processed/test-node1/4310907/2024-01-12T19-00-00/topdown-multianimal-id-133")
        if "AEON3" in str(root) else 
        Path(r"Z:/aeon/data/processed/test-node1/4350621/2024-01-22T19-00-00/topdown-multianimal-id-133")
    )

    # Create a `blocks` df with columns 'start', 'end', and 'root'
    blocks_df = pd.concat(
        [
         blocks_df,
         pd.DataFrame(
             {   
                 "root": [root] * len(block_start_times),
                 "sleap_model_dir": [sleap_model_dir] * len(block_start_times),
                 "start": block_start_times, 
                 "end": block_end_times, 
             }
        )
       ], ignore_index=True
    )

# Add columns to `blocks_df`
new_cols = [
    "block_duration",
    "subjects",  # list of subjects in block
    "patch_info",  # df: index: patch; cols: rate, offset
    "pellet_info",  # df: index: del ts; cols: patch, thresh, id {for each pel del, get last thresh}
    "cum_wheel_dist",  # DotMap: patch: df
]
for col in new_cols:
    blocks_df[col] = None

blocks_df = blocks_df.sort_values(by="start").reset_index(drop=True)
display(blocks_df)

C:\Users\jai\AppData\Local\Temp\ipykernel_24468\210757729.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if block_info.pellet_ct[i - 1] > 0:
C:\Users\jai\AppData\Local\Temp\ipykernel_24468\210757729.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif block_info.pellet_ct[i - 1] == 0 and block_info.index[i] - block_info.index[i - 1] > pd.Timedelta("1s"):
C:\Users\jai\AppData\Local\Temp\ipykernel_24468\210757729.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access 

,root,sleap_model_dir,start,end,block_duration,subjects,patch_info,pellet_info,cum_wheel_dist
0,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-01-31 12:59:06.005983829,2024-01-31 14:58:11.045983791,None,None,None,None,None
1,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-01-31 12:59:14.001984119,2024-01-31 14:45:59.000000000,None,None,None,None,None
2,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-01-31 14:45:55.000000000,2024-01-31 16:18:11.001984119,None,None,None,None,None
3,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-01-31 14:58:07.045983791,2024-01-31 17:49:26.000000000,None,None,None,None,None
4,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-01-31 16:18:07.001984119,2024-01-31 17:56:23.000000000,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...
75,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-02-03 09:41:16.000000000,2024-02-03 12:04:48.000000000,None,None,None,None,None
76,Z:\aeon\data\raw\AEON4\social0.2,Z:\aeon\data\processed\test-node1\4350621\2024...,2024-02-03 11:47:26.004000186,2024-02-03 14:11:22.001984119,None,None,None,None,None
77,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-02-03 12:04:44.000000000,2024-02-03 14:00:03.007999897,None,None,None,None,None
78,Z:\aeon\data\raw\AEON3\social0.2,Z:\aeon\data\processed\test-node1\4310907\2024...,2024-02-03 13:59:59.007999897,2024-02-03 15:43:05.005983829,None,None,None,None,None


In [5]:
"""Get subject env visits."""

subject_env_visits = {}

for root, arena in zip(roots, arenas):
    subject_visits = aeon.load(
        root, social02.Environment.SubjectVisits, pd.Timestamp("2024-01-31 00:00:00"), exp_end
    )
    # Find all rows where:
    #  - 'id' column starts with "*AA"
    #  - 'type' column is either "Enter" or "Exit" or "Remain",
    #  - 'region' column is "Environment"
    subject_visits = subject_visits[
        (subject_visits.id.str.contains("^.*AA"))
        & (subject_visits.type.isin(["Enter", "Exit", "Remain"]))
        & (subject_visits.region == "Environment")
    ]
    subject_env_visits[arena] = subject_visits

In [11]:
"""Fill out blocks df."""

warnings.filterwarnings("ignore")

skipped_blocks = np.full(len(blocks_df), False)

for block_i, block in tqdm(enumerate(blocks_df.itertuples()), desc="Processing blocks"):
    # if block_i < 158:
    #     continue
    # Compute block duration
    blocks_df.at[block.Index, "block_duration"] = block.end - block.start
    # <s Get subjects within the block:
    # Get all unique subjects that visited the environment over the entire exp;
    # For each subject, see 'type' of visit most recent to start of block;
    # If "Exit", this animal was not in the block.
    for arena in arenas:
        if arena in str(block.root):
            cur_env_visits = subject_env_visits[arena]
            break
    possible_subjects = cur_env_visits.id.unique().tolist()
    subjects = possible_subjects.copy()
    for subject in possible_subjects:
        subj_visit_in_time = np.logical_and(
            cur_env_visits.id == subject, cur_env_visits.index < block.start
        )
        if not np.any(subj_visit_in_time):  # if no subject visits prior to block start, drop it
            subjects.remove(subject)
        else:  # if visits, get most recent visit type before block; if "Exit", drop it
            pre_block_visit = cur_env_visits[subj_visit_in_time].iloc[-1]
            # last_visit = cur_env_visits[cur_env_visits.id == subject].iloc[-1]
            if pre_block_visit.type == "Exit":
                subjects.remove(subject)
    blocks_df.at[block.Index, "subjects"] = subjects
    # /s>
    # <s See if we should continue with analyzing this block
    cum_pel_ct = 0
    for patch in patches:
        r = eval(f"social02.{patch}.DepletionState")
        patch_df = aeon.load(block.root, r, block.start, block.end)
        cum_pel_ct += sum(np.diff(patch_df.index) > pd.Timedelta("1s"))
    if (cum_pel_ct < good_block_pel_ct) or (len(subjects) < 1):
        skipped_blocks[block_i] = True
        continue
    # /s>
    # <s Get pose-tracking info in order to do subject-specific assignments
    pose_df = aeon.load(block.root, social02.CameraTop.Pose, block.start, block.end)
    pose_df = reader.Pose.class_int2str(pose_df, block.sleap_model_dir)
    if len(subjects) == 1:  # fix mistaken sleap assignments for single-subject blocks
        pose_df["class"] = subjects[0]
    # /s>
    # <s Get per patch data (fill in `patch_info`, `cum_wheel_dist`, `pellet_info` cols of `blocks_df`)
    patch_stats_df = pd.DataFrame(index=patches, columns=["mean", "offset"])  # -> patch_info
    cum_wheel_dist_dm = DotMap()  # -> cum_wheel_dist
    pellets_stats_df = pd.DataFrame(columns=["time", "patch", "threshold", "id"])  # -> pellet_info
    for i, patch in enumerate(patches):
        # <ss Get wheel data
        r = eval(f"social02.{patch}.Encoder")
        wheel_df = aeon.load(block.root, r, block.start, block.end)[::50]
        cum_wheel_dist = -distancetravelled(wheel_df.angle)
        # /ss>
        # <ss Get pellets data
        r = eval(f"social02.{patch}.DepletionState")
        patch_df = aeon.load(block.root, r, block.start, block.end)
        rate, offset = patch_df[["rate", "offset"]].iloc[0]
        patch_stats_df.loc[patch, ["mean", "offset"]] = (1 / rate // 100 * 100, offset)
        patch_df_good_indxs = np.concatenate((np.diff(patch_df.index) > pd.Timedelta("1s"), (True,)))
        patch_df_for_pellets_df = patch_df[patch_df_good_indxs].reset_index()[["time", "threshold"]]
        patch_df_for_pellets_df["patch"] = patch
        patch_df_for_pellets_df["id"] = None
        patch_df_for_pellets_df.dropna(subset=["threshold"], inplace=True)
        # drop 1st val as is from block start
        patch_df_for_pellets_df = patch_df_for_pellets_df.iloc[1:].reset_index(drop=True)
        # /ss>
        # <ss Assign data to subjects
        if len(subjects) == 1:
            cum_wheel_dist_dm[patch] = cum_wheel_dist.to_frame(name=subjects[0])
            patch_df_for_pellets_df["id"] = subjects[0]
        else:
            # <sss Assign id based on which subject was closest to patch at time of event
            # <ssss Get distance-to-patch at each pose data timestep
            patch_xy = np.array(patch_locs[patch][arena]).astype(np.uint32)
            subjects_xy = pose_df[pose_df["part"] == "centroid"][["x", "y"]].values
            dist_to_patch = np.sqrt(np.sum((subjects_xy - patch_xy) ** 2, axis=1))
            dist_to_patch_df = pose_df[["class"]].copy()
            dist_to_patch_df["dist_to_patch"] = dist_to_patch
            # /ssss>
            # <ssss Get distance-to-patch at each wheel ts and pel del ts, organized by subject
            dist_to_patch_wheel_ts_id_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects)
            dist_to_patch_pel_ts_id_df = pd.DataFrame(
                index=patch_df_for_pellets_df["time"], columns=subjects
            )
            for subject in subjects:
                # Find closest match between pose_df indices and wheel indices
                dist_to_patch_wheel_ts_subj = pd.merge_asof(
                    left=dist_to_patch_wheel_ts_id_df[subject],
                    right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                    left_index=True,
                    right_index=True,
                    direction="forward",
                    tolerance=pd.Timedelta("100ms"),
                )
                dist_to_patch_wheel_ts_id_df[subject] = dist_to_patch_wheel_ts_subj["dist_to_patch"]
                # Find closest match between pose_df indices and pel indices
                dist_to_patch_pel_ts_subj = pd.merge_asof(
                    left=dist_to_patch_pel_ts_id_df[subject],
                    right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                    left_index=True,
                    right_index=True,
                    direction="forward",
                    tolerance=pd.Timedelta("200ms"),
                )
                dist_to_patch_pel_ts_id_df[subject] = dist_to_patch_pel_ts_subj["dist_to_patch"]
            # /ssss>
            # <ssss Get closest subject to patch at each pel del timestep
            patch_df_for_pellets_df["id"] = dist_to_patch_pel_ts_id_df.idxmin(axis=1).values
            # /ssss>
            # <ssss Get closest subject to patch at each wheel timestep
            cum_wheel_dist_subj_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects, data=0.0)
            closest_subjects = dist_to_patch_wheel_ts_id_df.idxmin(axis=1)
            wheel_dist = cum_wheel_dist.diff().fillna(cum_wheel_dist.iloc[0])
            # Assign wheel dist to closest subject for each wheel timestep
            for subject in subjects:
                subj_idxs = cum_wheel_dist_subj_df[closest_subjects == subject].index
                cum_wheel_dist_subj_df.loc[subj_idxs, subject] = wheel_dist[subj_idxs]
            cum_wheel_dist_dm[patch] = cum_wheel_dist_subj_df.cumsum(axis=0)
            # /ssss> #/sss> #/ss>
        pellets_stats_df = pd.concat([pellets_stats_df, patch_df_for_pellets_df], ignore_index=True)
        # /s>
    blocks_df.at[block.Index, "patch_info"] = patch_stats_df
    blocks_df.at[block.Index, "pellet_info"] = pellets_stats_df
    blocks_df.at[block.Index, "cum_wheel_dist"] = cum_wheel_dist_dm

Processing blocks: 0it [00:00, ?it/s]

Processing blocks: 80it [5:59:50, 269.89s/it]


In [24]:
"""Test individual block."""

block_idx = 23
block = list(blocks_df.itertuples())[block_idx]

# Compute block duration
blocks_df.at[block.Index, "block_duration"] = block.end - block.start
# <s Get subjects within the block:
# Get all unique subjects that visited the environment over the entire exp;
# For each subject, see 'type' of visit most recent to start of block;
# If "Exit", this animal was not in the block.
for arena in arenas:
    if arena in str(block.root):
        cur_env_visits = subject_env_visits[arena]
        break
possible_subjects = cur_env_visits.id.unique().tolist()
subjects = possible_subjects.copy()
for subject in possible_subjects:
    subj_visit_in_time = np.logical_and(
        cur_env_visits.id == subject, cur_env_visits.index < block.start
    )
    if not np.any(subj_visit_in_time):  # if no subject visits prior to block start, drop it
        subjects.remove(subject)
    else:  # if visits, get most recent visit type before block; if "Exit", drop it
        pre_block_visit = cur_env_visits[subj_visit_in_time].iloc[-1]
        # last_visit = cur_env_visits[cur_env_visits.id == subject].iloc[-1]
        if pre_block_visit.type == "Exit":
            subjects.remove(subject)
blocks_df.at[block.Index, "subjects"] = subjects
# /s>
# <s See if we should continue with analyzing this block
cum_pel_ct = 0
for patch in patches:
    r = eval(f"social02.{patch}.DepletionState")
    patch_df = aeon.load(block.root, r, block.start, block.end)
    cum_pel_ct += sum(np.diff(patch_df.index) > pd.Timedelta("1s"))
# if cum_pel_ct < good_block_pel_ct:
#     continue
# /s>
# <s Get pose-tracking info in order to do subject-specific assignments
pose_df = aeon.load(block.root, social02.CameraTop.Pose, block.start, block.end)
pose_df = reader.Pose.class_int2str(pose_df, block.sleap_model_dir)
if len(subjects) == 1:  # fix mistaken sleap assignments for single-subject blocks
    pose_df["class"] = subjects[0]
# /s>
# <s Get per patch data (fill in `patch_info`, `cum_wheel_dist`, `pellet_info` cols of `blocks_df`)
patch_stats_df = pd.DataFrame(index=patches, columns=["mean", "offset"])  # -> patch_info
cum_wheel_dist_dm = DotMap()  # -> cum_wheel_dist
pellets_stats_df = pd.DataFrame(columns=["time", "patch", "threshold", "id"])  # -> pellet_info
for i, patch in enumerate(patches):
    # <ss Get wheel data
    r = eval(f"social02.{patch}.Encoder")
    wheel_df = aeon.load(block.root, r, block.start, block.end)[::50].round(1).astype(np.float32)
    cum_wheel_dist = -distancetravelled(wheel_df.angle)
    # /ss>
    # <ss Get pellets data
    r = eval(f"social02.{patch}.DepletionState")
    patch_df = aeon.load(block.root, r, block.start, block.end)
    rate, offset = patch_df[["rate", "offset"]].iloc[0]
    patch_stats_df.loc[patch, ["mean", "offset"]] = (1 / rate // 100 * 100, offset)
    patch_df_good_indxs = np.concatenate((np.diff(patch_df.index) > pd.Timedelta("1s"), (True,)))
    patch_df_for_pellets_df = patch_df[patch_df_good_indxs].reset_index()[["time", "threshold"]]
    patch_df_for_pellets_df["patch"] = patch
    patch_df_for_pellets_df["id"] = None
    patch_df_for_pellets_df.dropna(subset=["threshold"], inplace=True)
    # drop 1st val as is from block start
    patch_df_for_pellets_df = patch_df_for_pellets_df.iloc[1:].reset_index(drop=True)
    # /ss>
    # <ss Assign data to subjects
    if len(subjects) == 1:
        cum_wheel_dist_dm[patch] = cum_wheel_dist.to_frame(name=subjects[0])
        patch_df_for_pellets_df["id"] = subjects[0]
    else:
        # <sss Assign id based on which subject was closest to patch at time of event
        # <ssss Get distance-to-patch at each pose data timestep
        patch_xy = np.array(patch_locs[patch][arena]).astype(np.uint32)
        subjects_xy = pose_df[pose_df["part"] == "centroid"][["x", "y"]].values
        dist_to_patch = np.sqrt(np.sum((subjects_xy - patch_xy) ** 2, axis=1))
        dist_to_patch_df = pose_df[["class"]].copy()
        dist_to_patch_df["dist_to_patch"] = dist_to_patch
        # /ssss>
        # <ssss Get distance-to-patch at each wheel ts and pel del ts, organized by subject
        dist_to_patch_wheel_ts_id_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects)
        dist_to_patch_pel_ts_id_df = pd.DataFrame(
            index=patch_df_for_pellets_df["time"], columns=subjects
        )
        for subject in subjects:
            # Find closest match between pose_df indices and wheel indices
            dist_to_patch_wheel_ts_subj = pd.merge_asof(
                left=dist_to_patch_wheel_ts_id_df[subject],
                right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                left_index=True,
                right_index=True,
                direction="forward",
                tolerance=pd.Timedelta("100ms"),
            )
            dist_to_patch_wheel_ts_id_df[subject] = dist_to_patch_wheel_ts_subj["dist_to_patch"]
            # Find closest match between pose_df indices and pel indices
            dist_to_patch_pel_ts_subj = pd.merge_asof(
                left=dist_to_patch_pel_ts_id_df[subject],
                right=dist_to_patch_df[dist_to_patch_df["class"] == subject],
                left_index=True,
                right_index=True,
                direction="forward",
                tolerance=pd.Timedelta("200ms"),
            )
            dist_to_patch_pel_ts_id_df[subject] = dist_to_patch_pel_ts_subj["dist_to_patch"]
        # /ssss>
        # <ssss Get closest subject to patch at each pel del timestep
        patch_df_for_pellets_df["id"] = dist_to_patch_pel_ts_id_df.idxmin(axis=1).values
        # /ssss>
        # <ssss Get closest subject to patch at each wheel timestep
        cum_wheel_dist_subj_df = pd.DataFrame(index=cum_wheel_dist.index, columns=subjects, data=0.)
        closest_subjects = dist_to_patch_wheel_ts_id_df.idxmin(axis=1)
        wheel_dist = cum_wheel_dist.diff().fillna(cum_wheel_dist.iloc[0])
        # Assign wheel dist to closest subject for each wheel timestep
        for subject in subjects:
            subj_idxs = cum_wheel_dist_subj_df[closest_subjects == subject].index
            cum_wheel_dist_subj_df.loc[subj_idxs, subject] = wheel_dist[subj_idxs]
        cum_wheel_dist_dm[patch] = cum_wheel_dist_subj_df.cumsum(axis=0)
        # /ssss> #/sss> #/ss>
    pellets_stats_df = pd.concat([pellets_stats_df, patch_df_for_pellets_df], ignore_index=True)
    # /s>
blocks_df.at[block.Index, "patch_info"] = patch_stats_df
blocks_df.at[block.Index, "pellet_info"] = pellets_stats_df
blocks_df.at[block.Index, "cum_wheel_dist"] = cum_wheel_dist_dm

---

## Scratchpad

In [ ]:
"""Get strange blocks."""

# Get blocks with no subjects
np.where(blocks_df.subjects.apply(len) == 0)

# Get blocks with < 3 pellets

In [4]:
blocks_df_early = pd.read_pickle(
    Path(r"Z:\aeon\code\scratchpad\jai\social02\social02_2024-01-31_2024-02-14.pkl")
)
blocks_df_late = pd.read_pickle(
    Path(r"Z:\aeon\code\scratchpad\jai\social02\social02_2024-02-14_2024-02-16.pkl")
)

In [151]:
# Co-foraging in general

# Get rfid beeps from all patches from 2024-02-12 00:00:00 to 2024-02-16 00:00:00
rfid_patch1 = aeon.load(
    roots[1], social02.Patch1Rfid.RfidEvents, pd.Timestamp("2024-02-12 00:00:00"), exp_end
)
rfid_patch1.columns = ["patch1"]
rfid_patch2 = aeon.load(
    roots[1], social02.Patch2Rfid.RfidEvents, pd.Timestamp("2024-02-12 00:00:00"), exp_end
)
rfid_patch2.columns = ["patch2"]
rfid_patch3 = aeon.load(
    roots[1], social02.Patch3Rfid.RfidEvents, pd.Timestamp("2024-02-12 00:00:00"), exp_end
)
rfid_patch3.columns = ["patch3"]
rfid_patch1 = rfid_patch1.map(lambda x: int(str(x)[-4:]) if pd.notnull(x) else x)
rfid_patch2 = rfid_patch2.map(lambda x: int(str(x)[-4:]) if pd.notnull(x) else x)
rfid_patch3 = rfid_patch3.map(lambda x: int(str(x)[-4:]) if pd.notnull(x) else x)
# Concatenate into one big df with patches as columns
all_rfid = pd.concat([rfid_patch1, rfid_patch2, rfid_patch3], axis=0)
# Group by subject
mask = all_rfid.apply(lambda x: (x == 9293).any(), axis=1)
baa48_beeps = all_rfid[mask]
mask = all_rfid.apply(lambda x: (x == 8675).any(), axis=1)
baa49_beeps = all_rfid[mask]
# Sort by index (time)
baa48_beeps = baa48_beeps.sort_index()
baa49_beeps = baa49_beeps.sort_index()
# For one subject's beeps, find the closest beep from another subject
# If within 3s, count as co-foraging
differences = np.abs(np.array(baa48_beeps.index)[:, None] - np.array(baa49_beeps.index))
coforaging_mask = differences < pd.Timedelta("3s")
sum_counts = np.zeros(coforaging_mask.shape[1])
for i in range(coforaging_mask.shape[1]):
    sum_counts[i] = np.sum(coforaging_mask[:, i])
coforaging_counts = len(np.where(sum_counts)[0])
# Co-foraging at specific patches

In [152]:
display(baa48_beeps)
display(baa49_beeps)

,patch1,patch2,patch3
time,,,
2024-02-12 01:40:14.648096085,NaN,NaN,9293.0
2024-02-12 01:40:15.172992229,NaN,NaN,9293.0
2024-02-12 01:40:15.571360111,NaN,NaN,9293.0
2024-02-12 01:40:16.091263771,NaN,NaN,9293.0
2024-02-12 01:40:16.489632130,NaN,NaN,9293.0
...,...,...,...
2024-02-15 22:36:43.095583916,9293.0,NaN,NaN
2024-02-15 22:36:43.678688049,9293.0,NaN,NaN
2024-02-15 22:36:44.077151775,9293.0,NaN,NaN


,patch1,patch2,patch3
time,,,
2024-02-12 07:47:16.309663773,NaN,8675.0,NaN
2024-02-12 07:47:16.894495964,NaN,8675.0,NaN
2024-02-12 07:47:17.258240223,NaN,8675.0,NaN
2024-02-12 07:47:17.717887878,NaN,8675.0,NaN
2024-02-12 07:47:17.938047886,NaN,8675.0,NaN
...,...,...,...
2024-02-15 22:24:52.164703846,8675.0,NaN,NaN
2024-02-15 22:24:52.839072227,8675.0,NaN,NaN
2024-02-15 22:24:53.268191813,8675.0,NaN,NaN


In [150]:
baa45_coforaging_rate = coforaging_counts / coforaging_mask.shape[0]
baa47_coforaging_rate = coforaging_counts / coforaging_mask.shape[1]

In [153]:
baa48_coforaging_rate = coforaging_counts / coforaging_mask.shape[0]
baa49_coforaging_rate = coforaging_counts / coforaging_mask.shape[1]

In [201]:
coforaging_patch_times = baa49_beeps.iloc[np.where(sum_counts)[0]]